# Performance Metrics

### Here we take our pretrained models (LightGBM, CNN, Autoencoder & Ensemble) and calculate the following
    
- Predictions +  MSE & MAE, Correlation (CORR)
  

0. Load Test Data

In [ ]:
from pyarrow.parquet import ParquetFile, ParquetDataset
import pyarrow as pa
import pandas as pd
import json

# Load datasets for next step
DATA_VERSION = 'v4.3'

try:
  import pandas as pd
  from google.colab import drive
  drive.mount('/content/drive')
  arrow_dataset = ParquetDataset('/content/drive/MyDrive/validation_52w.parquet')
  feature_metadata = json.load(open(f"/content/drive/MyDrive/features.json"))
  feature_set = feature_metadata["feature_sets"]['all']

  arrow_table = arrow_dataset.read()
  validation = arrow_table.to_pandas()
except:
  try:
    pf = ParquetFile('./v4.3/validation_int8.parquet')
    first_300k_rows = next(pf.iter_batches(batch_size = 300000))
    validation = pa.Table.from_batches([first_300k_rows]).to_pandas()
    validation = validation.filter(["era", "target"] + feature_set)
    validation["era_num"] = validation["era"].astype('int')
    validation = validation[validation['era_num'].isin(range(580,632))]
    feature_metadata = json.load(open(f"features.json"))
    feature_set = feature_metadata["feature_sets"]['all']
  except:
    validation = pd.read_parquet("validation_52w.parquet") # read a pre filtered copy
    feature_metadata = json.load(open(f"features.json"))
    feature_set = feature_metadata["feature_sets"]['all']
display(validation.head())
validation.shape

Mounted at /content/drive


,era,target,feature_aaronic_unexampled_arguer,feature_abactinal_inventable_luminescence,feature_abating_unadaptable_weakfish,feature_abdominal_subtriplicate_fin,feature_abducent_unbeneficed_lithophyte,feature_abducted_euphonic_pipewort,feature_ablest_mauritanian_elding,feature_abreast_viscoelastic_commander,...,feature_yokelish_metapsychological_lunt,feature_yorkist_authenticated_lotted,feature_yoruban_purplish_directoire,feature_yoruban_unapplied_tawse,feature_zincky_unseemly_butt,feature_zincoid_peccant_greywacke,feature_zoophoric_underglaze_algin,feature_zygodactyl_exponible_lathi,feature_zymotic_roundabout_figuration,era_num
id,,,,,,,,,,,,,,,,,,,,,
n0006f47ec6e34f7,0580,0.50,1,1,1,4,3,4,3,4,...,3,0,4,2,3,4,2,2,3,580
n001acdda0323855,0580,0.50,2,2,4,0,2,2,4,3,...,3,3,1,1,3,1,1,1,3,580
n002dd4f62cfab38,0580,0.50,4,1,1,1,2,3,1,3,...,4,3,2,0,2,2,4,4,0,580
n0035e776e3f1642,0580,0.00,3,0,4,2,4,2,1,3,...,2,2,4,0,4,2,1,2,1,580
n0048b94f2bcf186,0580,0.75,3,0,0,1,4,0,3,1,...,0,1,2,2,0,0,4,0,4,580


(258121, 2379)

1. LightGBM

In [ ]:
import lightgbm as lgb
#load from model:
lgbm = lgb.Booster(model_file='LGBM_Full.txt')

# Generate predictions against the out-of-sample validation features
# This will take a few minutes
validation["prediction_lgbm"] = lgbm.predict(validation[feature_set])
display(validation[["era", "prediction_lgbm", "target"]].head())

# Calculate MAE & MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_lgbm = mean_squared_error(validation["target"], validation['prediction_lgbm'])
mae_lgbm = mean_absolute_error(validation["target"], validation['prediction_lgbm'])

print(f'LightGBM: MAE = {mae_lgbm} | MSE = {mse_lgbm}')

,era,prediction_lgbm,target
id,,,
n0006f47ec6e34f7,0580,0.493688,0.50
n001acdda0323855,0580,0.510784,0.50
n002dd4f62cfab38,0580,0.507048,0.50
n0035e776e3f1642,0580,0.501867,0.00
n0048b94f2bcf186,0580,0.511338,0.75


LightGBM: MAE = 0.1525097411965613 | MSE = 0.04970784923358427


2. CNN

In [ ]:
!pip install torch --quiet

import torch
import torch.nn as nn

# Define CNN Class
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(num_features=32)
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.batch_norm2 = nn.BatchNorm1d(num_features=64)
        self.adaptive_avg_pool = nn.AdaptiveAvgPool1d(output_size=1)
        self.linear1 = nn.Linear(64, 1)


    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.batch_norm1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.batch_norm2(x)
        x = self.adaptive_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = torch.relu(x)

        return x

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

if device == 'cuda':
  cnn = torch.load("/content/cnn.pt")
else:
  cnn = torch.load("/content/cnn.pt", map_location=torch.device('cpu'))

cnn.eval()


# Ensure the model is in the evaluation mode
from torch.utils.data import TensorDataset, DataLoader

# Prepare the validation dataset
X_val_tensor = torch.Tensor(validation[feature_set].values)
y_val_tensor = torch.Tensor(validation['target'].values)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

## Make Predictions
all_predictions = []
all_targets = []
if device == 'cuda':
  use_gpu = 1
else:
  use_gpu = 0
with torch.no_grad():
    for features, targets in val_dataloader:
      if use_gpu:
          features = features.cuda()
          targets = targets.cuda()
      features = features.unsqueeze(1)
      targets = targets.unsqueeze(1)
      predictions = cnn(features)
      all_predictions.extend(predictions.cpu().numpy())
      all_targets.extend(targets.cpu().numpy())

# Append predictions to validation df
import numpy as np
if all(np.array(all_targets)[:100].reshape(100,) == np.array((validation['target']))[:100].reshape(100,)): # Checking if indexes match by comparing actual and expected targets
  validation['prediction_cnn'] = np.array(all_predictions).reshape(len(all_predictions),).tolist()

# Calculate MAE & MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_cnn = mean_squared_error(validation["target"], validation['prediction_cnn'])
mae_cnn = mean_absolute_error(validation["target"], validation['prediction_cnn'])
display(validation[["era", "prediction_cnn", "target"]].head())

print(f'CNN: MAE = {mae_cnn} | MSE = {mse_cnn}')

,era,prediction_cnn,target
id,,,
n0006f47ec6e34f7,0580,0.151624,0.50
n001acdda0323855,0580,0.125771,0.50
n002dd4f62cfab38,0580,0.151836,0.50
n0035e776e3f1642,0580,0.135151,0.00
n0048b94f2bcf186,0580,0.156545,0.75


CNN: MAE = 0.35771421747975934 | MSE = 0.16684877260046319


3. Autoencoder

In [ ]:
import pandas as pd
import json
if 'validation' not in globals():
  validation = pd.read_parquet("validation_52w.parquet") # read a pre filtered copy
  feature_metadata = json.load(open(f"features.json"))
  feature_set = feature_metadata["feature_sets"]['all']


In [ ]:
import torch
import torch.nn as nn

# Define the Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, feature_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(feature_dim, 1500),
            nn.BatchNorm1d(1500),
            Swish(),
            nn.Linear(1500, 1000),
            nn.BatchNorm1d(1000),
            Swish(),
            nn.Linear(1000, 500),
            nn.BatchNorm1d(500),
            Swish(),
            nn.Linear(500, encoding_dim),
            nn.BatchNorm1d(encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 500),
            nn.BatchNorm1d(500),
            Swish(),
            nn.Linear(500, 1000),
            nn.BatchNorm1d(1000),
            Swish(),
            nn.Linear(1000, 1500),
            nn.BatchNorm1d(1500),
            Swish(),
            nn.Linear(1500, feature_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Define the MLP
class MLP(nn.Module):
    def __init__(self, input_dim, dropout_rate):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Load Models
dropout_rate = 0.4
feature_dim = 2376  # Your feature dimension
encoding_dim = 1000  # Your encoding dimension
input_dim = feature_dim + encoding_dim  # Total input dimension for MLP

autoencoder = Autoencoder(feature_dim, encoding_dim)
mlp = MLP(input_dim, dropout_rate)

autoencoder.load_state_dict(torch.load('autoencoder.pth')['autoencoder'])
mlp.load_state_dict(torch.load('autoencoder.pth')['mlp'])

autoencoder.eval()
mlp.eval()

# Ensure the model is in the evaluation mode
from torch.utils.data import TensorDataset, DataLoader

# Prepare the validation dataset
X_val_tensor = torch.tensor(validation[feature_set].values, dtype=torch.float32)
y_val_tensor = torch.tensor(validation['target'].values, dtype=torch.float32).view(-1, 1)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

## Make Predictions
all_predictions = []
all_targets = []

with torch.no_grad():
    for data, targets in val_dataloader:
        encoded, _ = autoencoder(data)  # Only encoded is needed for prediction
        predictions = mlp(torch.cat((encoded, data), dim=1))
        all_predictions.extend(predictions.numpy())
        all_targets.extend(targets.numpy())

# Append predictions to validation df
import numpy as np
if all(np.array(all_targets)[:100].reshape(100,) == np.array((validation['target']))[:100].reshape(100,)): # Checking if indexes match by comparing actual and expected targets
  validation['prediction_autoencoder'] = np.array(all_predictions).reshape(len(all_predictions),).tolist()

# Calculate MAE & MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_auto = mean_squared_error(validation["target"], validation['prediction_autoencoder'])
mae_auto = mean_absolute_error(validation["target"], validation['prediction_autoencoder'])
display(validation[["era", "prediction_autoencoder", "target"]].head())

print(f'Autoencoder + MLP : MAE = {mae_auto} | MSE = {mse_auto}')

,era,prediction_autoencoder,target
id,,,
n0006f47ec6e34f7,0580,0.493854,0.50
n001acdda0323855,0580,0.527802,0.50
n002dd4f62cfab38,0580,0.511937,0.50
n0035e776e3f1642,0580,0.509761,0.00
n0048b94f2bcf186,0580,0.505938,0.75


Autoencoder + MLP : MAE = 0.15413900131608357 | MSE = 0.049752629992081014


4. Ensemble

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import lightgbm as lgb
import json
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Define the Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, feature_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(feature_dim, 1500),
            nn.BatchNorm1d(1500),
            Swish(),
            nn.Linear(1500, 1000),
            nn.BatchNorm1d(1000),
            Swish(),
            nn.Linear(1000, 500),
            nn.BatchNorm1d(500),
            Swish(),
            nn.Linear(500, encoding_dim),
            nn.BatchNorm1d(encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 500),
            nn.BatchNorm1d(500),
            Swish(),
            nn.Linear(500, 1000),
            nn.BatchNorm1d(1000),
            Swish(),
            nn.Linear(1000, 1500),
            nn.BatchNorm1d(1500),
            Swish(),
            nn.Linear(1500, feature_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Define the MLP
class MLP(nn.Module):
    def __init__(self, input_dim, dropout_rate):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.layers(x)

import torch
import torch.nn as nn

# Load Models
dropout_rate = 0.4
feature_dim = 2376  # Your feature dimension
encoding_dim = 1000  # Your encoding dimension
input_dim = feature_dim + encoding_dim  # Total input dimension for MLP

autoencoder = Autoencoder(feature_dim, encoding_dim)
mlp = MLP(input_dim, dropout_rate)

autoencoder.load_state_dict(torch.load('autoencoder.pth')['autoencoder'])
mlp.load_state_dict(torch.load('autoencoder.pth')['mlp'])

autoencoder.eval()
mlp.eval()

#load from model:
lgb = lgb.Booster(model_file='LGBM_Full.txt')

# Define your prediction pipeline as a function
def predict_ensemble(live_features: pd.DataFrame) -> pd.DataFrame:
    # #1 - LGBM

    lgb_predictions = lgb.predict(live_features[feature_set])
    lgb_predictions = pd.Series(lgb_predictions, index=live_features.index)

    # #2 - Autoencoder

    # Prepare the validation dataset
    X_val_tensor = torch.tensor(live_features[feature_set].values, dtype=torch.float32)
    y_val_tensor = torch.tensor(live_features['target'].values, dtype=torch.float32).view(-1, 1)

    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

    ## Make Predictions
    all_predictions = []

    with torch.no_grad():
        for data, _ in val_dataloader:
            encoded, _ = autoencoder(data)  # Only encoded is needed for prediction
            predictions = mlp(torch.cat((encoded, data), dim=1))
            all_predictions.extend(predictions.numpy())

    live_predictions = np.array(all_predictions).reshape(len(all_predictions),).tolist()

    auto_predictions = pd.Series(live_predictions, index=live_features.index)
    submission = lgb_predictions.add(auto_predictions, fill_value=0.5)/2

    return submission.to_frame("prediction")

# run pipeline and get predictions
predictions_ensemble = predict_ensemble(validation)

In [ ]:
# Append predictions to validation df
import numpy as np
validation['prediction_ensemble'] = predictions_ensemble['prediction']

# Calculate MAE & MSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_auto = mean_squared_error(validation["target"], validation['prediction_ensemble'])
mae_auto = mean_absolute_error(validation["target"], validation['prediction_ensemble'])
display(validation[["era", "prediction_ensemble", "target"]].head())

print(f'Ensemble: MAE = {mae_auto} | MSE = {mse_auto}')

,era,prediction_ensemble,target
id,,,
n0006f47ec6e34f7,0580,0.493771,0.50
n001acdda0323855,0580,0.519293,0.50
n002dd4f62cfab38,0580,0.509493,0.50
n0035e776e3f1642,0580,0.505814,0.00
n0048b94f2bcf186,0580,0.508638,0.75


Ensemble: MAE = 0.1529764298931693 | MSE = 0.04970235576228966


Summary

In [ ]:
validation[['era', 'target', 'prediction_lgbm','prediction_cnn','prediction_autoencoder', 'prediction_ensemble']].head()

,era,target,prediction_lgbm,prediction_cnn,prediction_autoencoder,prediction_ensemble
id,,,,,,
n0006f47ec6e34f7,0580,0.50,0.493688,0.50005,0.493854,0.493771
n001acdda0323855,0580,0.50,0.510784,0.50005,0.527802,0.519293
n002dd4f62cfab38,0580,0.50,0.507048,0.50005,0.511937,0.509493
n0035e776e3f1642,0580,0.00,0.501867,0.50005,0.509761,0.505814
n0048b94f2bcf186,0580,0.75,0.511338,0.50005,0.505938,0.508638


4. Charts

In [ ]:
# save predictions
# results.to_parquet('/content/drive/MyDrive/predictions.parquet')
# back = results.loc[:]

In [ ]:
# Import Results

if 'validation' in globals():
  try:
    results = validation.filter(['era', 'target', 'prediction_lgbm','prediction_cnn','prediction_autoencoder','prediction_ensemble'])
  except:
    import pandas as pd
    from google.colab import drive
    drive.mount('/content/drive')
    results = pd.read_parquet('/content/drive/MyDrive/predictions.parquet')

if 'validation' not in globals():
  import pandas as pd
  from google.colab import drive
  drive.mount('/content/drive')
  results = pd.read_parquet('/content/drive/MyDrive/predictions.parquet')

# Download Meta Model
# metamodel = pd.read_parquet(f"meta_model.parquet")
results.head()

,era,target,prediction_lgbm,prediction_cnn,prediction_autoencoder,prediction_ensemble
id,,,,,,
n0006f47ec6e34f7,0580,0.50,0.493688,0.151624,0.493854,0.493771
n001acdda0323855,0580,0.50,0.510784,0.125771,0.527802,0.519293
n002dd4f62cfab38,0580,0.50,0.507048,0.151836,0.511937,0.509493
n0035e776e3f1642,0580,0.00,0.501867,0.135151,0.509761,0.505814
n0048b94f2bcf186,0580,0.75,0.511338,0.156545,0.505938,0.508638
...,...,...,...,...,...,...
nffdaf81ad4a3d54,0631,0.50,0.494956,0.140483,0.500732,0.497844
nffe3a2c92e9e555,0631,0.25,0.498757,0.146570,0.510846,0.504802
nffe7d9ce7819521,0631,0.75,0.496236,0.158590,0.501658,0.498947


In [ ]:
# Calculate per era loss metrics
!pip install -q --no-deps numerai-tools
from numerai_tools.scoring import numerai_corr, correlation_contribution
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import plotly.express as px


# Calculate Errors
results['error_lgbm'] = abs(results['prediction_lgbm'] - results['target'])
results['error_cnn'] = abs(results['prediction_cnn'] - results['target'])
results['error_autoencoder'] = abs(results['prediction_autoencoder'] - results['target'])
results['error_ensemble'] = abs(results['prediction_ensemble'] - results['target'])

# Calculate Metrics by Era
results_mae = results.groupby('era')[['error_lgbm',	'error_cnn', 'error_autoencoder', 'error_ensemble']].agg('mean').reset_index()
results_mae = results_mae.rename(columns={'error_lgbm':'LightGBM',	'error_cnn':"CNN", 'error_autoencoder':"Autoencoder + MLP",'error_ensemble':"Ensemble (LGBM + Autoencoder)"})
def agg_mse(df):
    mse = np.sum(df**2)
    return mse/len(df)

results_mse = results.groupby('era')[['error_lgbm',	'error_cnn', 'error_autoencoder', 'error_ensemble']].apply(agg_mse).reset_index()
results_mse = results_mse.rename(columns={'error_lgbm':'LightGBM',	'error_cnn':"CNN", 'error_autoencoder':"Autoencoder + MLP",'error_ensemble':"Ensemble (LGBM + Autoencoder)"})

results_corr = results.groupby("era").apply(
    lambda x: numerai_corr(x[['prediction_lgbm','prediction_cnn','prediction_autoencoder','prediction_ensemble']].dropna(), x["target"].dropna())
)
results_corr = results_corr.reset_index()


### MAE and MSE by ERA

In [ ]:
1-0.151624
1-0.135151
1-0.156545
1-0.146570
1-0.158590

0.85343

In [ ]:
# MAE per era
px.line(data_frame = results_mae, x='era', y=['LightGBM', 'Autoencoder + MLP', 'Ensemble (LGBM + Autoencoder)'],
        labels={
                     "era": "ERA",
                     "value": "MEAN ABSOLUTE ERROR"
                 },
                title="MAE by Model (1 Year)"
                )


In [ ]:
px.line(data_frame = results_mae, x='era', y='CNN',
        labels={
                     "era": "ERA",
                     "value": "MEAN ABSOLUTE ERROR"
                 },
                title="MAE for CNN (1 Year)"
                )


In [ ]:
# MSE per era
px.line(data_frame = results_mse, x='era', y=['LightGBM', 'Autoencoder + MLP', 'Ensemble (LGBM + Autoencoder)'],
        labels={
                     "era": "ERA",
                     "value": "MEAN SQUARED ERROR"
                 },
                title="MSE by Model (1 Year)"
                )



In [ ]:
px.line(data_frame = results_mse, x='era', y='CNN',
        labels={
                     "era": "ERA",
                     "value": "MEAN ABSOLUTE ERROR"
                 },
                title="MSE for CNN (1 Year)"
                )


In [ ]:

# Plot Cumulative MAE Loss
results_mae['error_lgbm_cumsum'] = results_mae['LightGBM'].cumsum()
results_mae['error_cnn_cumsum'] = results_mae['CNN'].cumsum()
results_mae['error_autoencoder_cumsum'] = results_mae['Autoencoder + MLP'].cumsum()
results_mae['error_ensemble_cumsum'] = results_mae['Ensemble (LGBM + Autoencoder)'].cumsum()
px.line(data_frame = results_mae, x='era', y=['LightGBM','CNN', 'Autoencoder + MLP', 'Ensemble (LGBM + Autoencoder)'])


In [ ]:
# Cumulative MSE loss



px.line(data_frame = results_mse, x='era', y=['LightGBM','CNN', 'Autoencoder + MLP', 'Ensemble (LGBM + Autoencoder)'])


### Correlation by Era

In [ ]:
# Corr per era
px.line(data_frame = results_corr, x='era', y=['prediction_lgbm', 'prediction_cnn','prediction_autoencoder',	'prediction_ensemble'])


In [ ]:

# Plot Cumulative Corr
results_corr['lgbm_cumsum'] = abs(results_corr['prediction_lgbm']).cumsum()
results_corr['autoencoder_cumsum'] = abs(results_corr['prediction_autoencoder']).cumsum()
results_corr['ensemble_cumsum'] = results_corr['prediction_ensemble'].cumsum()
results_corr['cnn_cumsum'] = results_corr['prediction_cnn'].cumsum()

# results_corr = results_corr.reset_index()
px.line(data_frame = results_corr, x='era', y=['lgbm_cumsum', 'cnn_cumsum','autoencoder_cumsum', 'ensemble_cumsum'],
        labels={
                     "era": "ERA",
                     "value": "CUMULATIVE CORRELATION"
                 },
                title="Cumulative Correlation Score by Model (1 Year)")


### NumerAI Diagnostics (Backtest)
Link to our models on NumerAI: https://numer.ai/~rd_ (takes ~3 weeks for preliminary results to be generated)

![LGBM](https://drive.google.com/uc?export=view&id=1yS_czanuCuNHTcx-0FcDnOfXfdgY81P_)
![CNN](https://drive.google.com/uc?export=view&id=1bjeoLtVHXedKj1jaQMWuIQ9_EryD250A)
![Autoencoder](https://drive.google.com/uc?export=view&id=1QhM-3IwrdrkuSsPB6ET9PyanfNXGy_B2)

BEST

![Ensemble](https://drive.google.com/uc?export=view&id=107sMJA8zPRYvB7K49gXvi32bl9C-gWdm)
